In [1]:
import time
from itertools import product, combinations

import numpy as np
import pandas as pd
import gurobipy as gb
from sklearn.linear_model import LinearRegression


# WLS credentials
WLSACCESSID = 'ccc2c36a-db14-4956-b2e3-60adc45e9957'
WLSSECRET = '1e0e3dbf-7933-44dc-8f81-e0482ded7ac8'
LICENSEID = 2586688

# Create the Gurobi environment with parameters
env = gb.Env(empty=True)  # Start with an empty environment
env.setParam('WLSACCESSID', WLSACCESSID)
env.setParam('WLSSECRET', WLSSECRET)
env.setParam('LICENSEID', LICENSEID)
env.start() 


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2586688
Academic license 2586688 - for non-commercial use only - registered to ru___@ucsd.edu


<gurobipy.Env, Parameter changes: WLSAccessID=(user-defined), WLSSecret=(user-defined), LicenseID=2586688>

In [2]:
df = pd.read_csv('features.csv')
df.columns

Index(['Unnamed: 0', 'ap_ib', 'calculus', 'counselors', 'frpl_rate',
       'frac_sat_act', 'n_sat_act', 'n_A_m', 'n_A_f', 'n_B_m', 'n_B_f',
       'n_C_m', 'n_C_f', 'n_D_m', 'n_D_f', 'n_E_m', 'n_E_f', 'n_F_m', 'n_F_f',
       'n_G_m', 'n_G_f', 'n_A', 'n_B', 'n_C', 'n_D', 'n_E', 'n_F', 'n_G',
       'frac_A_m', 'frac_A_f', 'frac_B_m', 'frac_B_f', 'frac_C_m', 'frac_C_f',
       'frac_D_m', 'frac_D_f', 'frac_E_m', 'frac_E_f', 'frac_F_m', 'frac_F_f',
       'frac_G_m', 'frac_G_f', 'frac_A', 'frac_B', 'frac_C', 'frac_D',
       'frac_E', 'frac_F', 'frac_G', 'n_sat_act_A_m', 'n_sat_act_A_f',
       'n_sat_act_B_m', 'n_sat_act_B_f', 'n_sat_act_C_m', 'n_sat_act_C_f',
       'n_sat_act_D_m', 'n_sat_act_D_f', 'n_sat_act_E_m', 'n_sat_act_E_f',
       'n_sat_act_F_m', 'n_sat_act_F_f', 'n_sat_act_G_m', 'n_sat_act_G_f',
       'n_sat_act_A', 'n_sat_act_B', 'n_sat_act_C', 'n_sat_act_D',
       'n_sat_act_E', 'n_sat_act_F', 'n_sat_act_G', 'frac_sat_act_A_m',
       'frac_sat_act_A_f', 'frac_sat_act_B

# Explanation of Dataset Features

### General Information
- **`Unnamed: 0`**: Likely an index column automatically generated during data import. If not meaningful, it can be dropped.
- **`latitude`**: Latitude of the school, used for geographic analysis.
- **`longitude`**: Longitude of the school, used for geographic analysis.

---

### Input Features (`X`)
These represent characteristics of schools that may affect student outcomes:
- **`ap_ib`**: Indicator or count of students enrolled in Advanced Placement (AP) or International Baccalaureate (IB) programs. Higher values indicate better academic resources or rigor.
- **`calculus`**: Indicator or count of students enrolled in Calculus courses, which may act as a proxy for advanced math preparation.
- **`counselors`**: Number of counselors available at the school, potentially influencing college readiness and student support.
- **`frpl_rate`**: Percentage of students eligible for Free or Reduced-Price Lunch (FRPL), a socioeconomic indicator where higher values suggest greater economic disadvantage.

---

### Outcome Variables (`y`)
These represent the target outcomes or results that the model aims to improve:
- **`frac_sat_act`**: Fraction of students who took the SAT or ACT, a measure of college readiness.
- **`n_sat_act`**: Count of students who took the SAT or ACT.

---

### Demographic-Specific Counts (`n_*`)
These represent the **count of students** in specific demographic categories:
- **By Gender and Category (e.g., `n_A_m`, `n_A_f`)**:
  - `n_A_m`: Number of male students in demographic category A.
  - `n_A_f`: Number of female students in demographic category A.
  - Categories B through G follow the same format.
- **Aggregated by Category (e.g., `n_A`)**:
  - Total number of students in category A, regardless of gender.
  - Categories B through G are aggregated similarly.

---

### Demographic-Specific Fractions (`frac_*`)
These represent the **proportion of students** in specific demographic categories:
- **By Gender and Category (e.g., `frac_A_m`, `frac_A_f`)**:
  - `frac_A_m`: Fraction of male students in demographic category A.
  - `frac_A_f`: Fraction of female students in demographic category A.
  - Categories B through G follow the same format.
- **Aggregated by Category (e.g., `frac_A`)**:
  - Total fraction of students in category A, regardless of gender.
  - Categories B through G are aggregated similarly.

---

### SAT/ACT-Specific Counts and Fractions
These measure SAT/ACT participation within specific demographic categories:

#### Counts (`n_sat_act_*`):
- **By Gender and Category** (e.g., `n_sat_act_A_m`):
  - Count of male students in category A who took the SAT/ACT.
- **Aggregated by Category** (e.g., `n_sat_act_A`):
  - Total count of students in category A who took the SAT/ACT.
- Categories B through G follow the same format.

#### Fractions (`frac_sat_act_*`):
- **By Gender and Category** (e.g., `frac_sat_act_A_m`):
  - Fraction of male students in category A who took the SAT/ACT.
- **Aggregated by Category** (e.g., `frac_sat_act_A`):
  - Total fraction of students in category A who took the SAT/ACT.
- Categories B through G follow the same format.

---

### Other Features
- **`total_students`**: Total number of students in the school, regardless of demographic categories. Useful for normalizing counts or computing participation rates.

---

### Summary of Feature Groups

| **Feature Group**             | **Description**                                               |
|-------------------------------|-------------------------------------------------------------|
| General Information           | School index, latitude, longitude                           |
| Input Features                | Academic resources, socioeconomic data (`ap_ib`, `frpl_rate`) |
| Outcome Variables             | SAT/ACT participation metrics                               |
| Demographic Counts (`n_*`)    | Counts of students by demographic category and gender       |
| Demographic Fractions (`frac_*`) | Proportions of students by demographic category and gender  |
| SAT/ACT Counts and Fractions  | Participation counts and fractions by demographic group     |

---


In [3]:
# Define Constants
SOCIAL_CATEGORIES = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
BUDGET = 100
TAU_VALUES = [0.566, None]  # Define fairness constraints for optimization

# Data Preparation
df = pd.read_csv('features.csv')
X_columns = ['frpl_rate', 'calculus', 'ap_ib', 'counselors']
count_columns = [f'n_{category}' for category in SOCIAL_CATEGORIES]
frac_columns = [f'frac_{category}' for category in SOCIAL_CATEGORIES]

X = df[X_columns]
A_frac = df[frac_columns]
y_train = df['frac_sat_act'].values

neighbor_distance_matrix = np.load('neighbor_distance_matrix.npy')
neighbor_index_matrix = np.load('neighbor_index_matrix.npy')

ap_ib = X['ap_ib'].values
calculus = X['calculus'].values
counselors = X['counselors'].values
n = len(X)

In [4]:
X.shape

(490, 4)

In [5]:
AP_IB = X['ap_ib'].values
COUNSELORS = X['counselors'].values
FRPL = np.ones_like(X['frpl_rate'].values)
A_FRAC = df[frac_columns]
A_MATRIX = A_FRAC.values


NEIGHBOR_INDEX_MATRIX = np.load('neighbor_index_matrix.npy')
NEIGHBOR_DISTANCE_MATRIX = np.load('neighbor_distance_matrix.npy')
NUM_SCHOOLS = X.shape[0]
# weight_df = pd.read_csv('params_7_disagg.csv', index_col=0)
# WEIGHT_MATRIX = weight_df.values

#possible intervention - column represents neighbours
NUM_NEIGHBORS = NEIGHBOR_INDEX_MATRIX.shape[1]
intervention_sample_spaces = [(0, 1)] * NUM_NEIGHBORS
POSSIBLE_INTERVENTIONS_MATRIX = np.array(list(
    product(*intervention_sample_spaces)
))
NUM_POSSIBLE_INTERVENTIONS = POSSIBLE_INTERVENTIONS_MATRIX.shape[0]

BUDGET = 100

NUM_CATEGORIES = 28
CATEGORIES = list(range(NUM_CATEGORIES))
CATEGORY_PAIRS = list(combinations(CATEGORIES, 2))

DEMOGRAPHIC_COUNTERFACTUALS = [0, 1]
NUM_COUNTERFACTUALS = len(DEMOGRAPHIC_COUNTERFACTUALS)

TOTAL_STUDENTS = df['total_students'].values
R_COUNTS = df[count_columns].values
R_COUNTS_TOTAL = R_COUNTS.sum(axis=0)

CALCULUS = X['calculus']
A_DIMENSION = A_MATRIX.shape[1]

WHETHER_OR_NOT_CALCULUS_GIVEN_INTERFERENCE = np.max(
    NEIGHBOR_DISTANCE_MATRIX * CALCULUS.values, axis=1)

In [6]:
neighbor_index_matrix = np.load('neighbor_index_matrix.npy')
neighbor_distance_matrix = np.load('neighbor_distance_matrix.npy')

# Print basic information
print("Neighbor Index Matrix:")
print("Shape:", neighbor_index_matrix.shape)
print("Content (First 5 Rows):\n", neighbor_index_matrix[:5])

print("\nNeighbor Distance Matrix:")
print("Shape:", neighbor_distance_matrix.shape)
print("Content (First 5 Rows):\n", neighbor_distance_matrix[:5])

Neighbor Index Matrix:
Shape: (490, 6)
Content (First 5 Rows):
 [[  0 330 230 135 459 136]
 [  1 396 127 273 293 279]
 [  2 226 229 418  14  82]
 [  3 356  77  61  43 234]
 [  4 252 335 300 438 232]]

Neighbor Distance Matrix:
Shape: (490, 490)
Content (First 5 Rows):
 [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [7]:
neighbor_distance_matrix.sum()

805.5861871934077

In [8]:
# Get the lower triangular part of the matrix (excluding diagonal)
lower_triangle = np.tril(neighbor_distance_matrix, k=-1)

# Check if all elements are zero
if np.all(lower_triangle == 0):
    print("The matrix is an upper triangular matrix.")
else:
    print("The matrix is not an upper triangular matrix.")


The matrix is not an upper triangular matrix.


In [9]:
# Find and analyze differences
diff_indices = np.where(neighbor_distance_matrix != neighbor_distance_matrix.T)
for i, j in zip(diff_indices[0], diff_indices[1]):
    print(f"M[{i}, {j}] = {neighbor_distance_matrix[i, j]}, M[{j}, {i}] = {neighbor_distance_matrix[j, i]}")


M[0, 9] = 0.0, M[9, 0] = 0.0017915411464302121
M[0, 124] = 0.0, M[124, 0] = 0.0005530683345420044
M[0, 135] = 0.001921847232164395, M[135, 0] = 0.0
M[0, 283] = 0.0, M[283, 0] = 0.0016028179137796647
M[0, 344] = 0.0, M[344, 0] = 0.0008084983255359936
M[0, 433] = 0.0, M[433, 0] = 0.0016774575497498943
M[1, 273] = 0.0007760527447971058, M[273, 1] = 0.0
M[1, 279] = 0.0005143675875279728, M[279, 1] = 0.0
M[1, 292] = 0.0, M[292, 1] = 0.00035615200884159953
M[2, 96] = 0.0, M[96, 2] = 0.0005727412021071462
M[2, 153] = 0.0, M[153, 2] = 0.00030383884676690894
M[2, 324] = 0.0, M[324, 2] = 0.0005216299380675325
M[3, 43] = 0.0006285219191818083, M[43, 3] = 0.0
M[3, 57] = 0.0, M[57, 3] = 0.00044023567760300813
M[3, 61] = 0.00062968738546578, M[61, 3] = 0.0
M[3, 200] = 0.0, M[200, 3] = 0.00043993966451961775
M[3, 234] = 0.0006281841414887097, M[234, 3] = 0.0
M[4, 232] = 0.0005235882570458226, M[232, 4] = 0.0
M[4, 252] = 0.0007995374867753347, M[252, 4] = 0.0
M[4, 300] = 0.0005320336730580405, M[300, 

In [10]:
neighbor_distance_matrix[0, 135]

0.001921847232164395

In [11]:
# Calculate adjusted features for regression model
def compute_adjusted_features(feature_values, A_frac, neighbor_distance_matrix):
    max_neighbor_influence = np.max(neighbor_distance_matrix * feature_values.T, axis=1).reshape(n, 1)
    return A_frac * max_neighbor_influence

a_max_Sij_Pj = compute_adjusted_features(ap_ib, A_frac, neighbor_distance_matrix)
a_max_Sij_Cj = compute_adjusted_features(calculus, A_frac, neighbor_distance_matrix)
a_Fj = A_frac * counselors.reshape(n, 1)

# Combine features for regression model
X_train = np.concatenate((a_max_Sij_Pj, a_max_Sij_Cj, a_Fj, A_frac), axis=1)

# Train linear regression model
linmod = LinearRegression(fit_intercept=False).fit(X_train, y_train)
model_weights = linmod.coef_
param_dims = len(SOCIAL_CATEGORIES)

# Extract regression weights
weight_dict = {
    'alpha': model_weights[param_dims:param_dims*2],
    'beta': model_weights[:param_dims],
    'gamma': model_weights[param_dims*2:param_dims*3],
    'theta': model_weights[-param_dims:]
}
params = pd.DataFrame(weight_dict)

ALPHA, BETA, GAMMA, THETA = (params['alpha'].values, params['beta'].values, 
                             params['gamma'].values, params['theta'].values)

ALPHA, BETA, GAMMA, THETA

(array([-0.06676548,  0.02920318,  0.27690293, -0.1496839 ,  0.6896188 ,
         1.84418092, -0.37517525]),
 array([ 0.14725533,  0.11735532, -0.16410716,  0.05501886, -1.68741278,
        -0.61907302, -3.03710195]),
 array([ 0.00873755, -0.0087489 , -0.0043426 ,  0.01557128, -0.30437838,
        -0.05114738,  0.0774586 ]),
 array([ 0.09178179,  0.13514781,  0.21394424,  0.46154045,  3.30620549,
        -0.65855919,  4.18574581]))

In [12]:
# Optimization Helpers
def calculate_expected_impact(index, intervention_array, demographic_vector):
    nearest_neighbors = neighbor_index_matrix[index, :]
    neighbor_distances = neighbor_distance_matrix[index, nearest_neighbors]

    calculus_term = np.dot(demographic_vector, ALPHA) * np.max(neighbor_distances * intervention_array)
    ap_ib_term = np.dot(demographic_vector, BETA) * np.max(neighbor_distances * ap_ib[nearest_neighbors])
    counselors_term = np.dot(demographic_vector, GAMMA) * counselors[index]
    race_term = np.dot(demographic_vector, THETA)

    impact = calculus_term + ap_ib_term + counselors_term + race_term
    return max(min(impact, 1), 0)

In [13]:
def calculate_all_possible_impacts(index, demographic_vector, POSSIBLE_INTERVENTIONS_MATRIX):
    possible_impacts = np.empty(len(POSSIBLE_INTERVENTIONS_MATRIX))
    for k, intervention_array in enumerate(POSSIBLE_INTERVENTIONS_MATRIX):
        possible_impacts[k] = calculate_expected_impact(index, intervention_array, demographic_vector)
    return possible_impacts

In [14]:
# Optimization Routine
def optimize_interventions(tau_value, A_frac, POSSIBLE_INTERVENTIONS_MATRIX):
    print(f'Running optimization for tau={tau_value}')
    model = gb.Model(env=env)

    interventions = model.addVars(n, vtype=gb.GRB.BINARY, name="interventions")
    model.addConstr(sum(interventions.values()) <= BUDGET, "budget_constraint")

    def add_auxiliary_constraints(index):
        demographic_vector = A_frac.values[index, :]
        factual_impacts = calculate_all_possible_impacts(index, demographic_vector, POSSIBLE_INTERVENTIONS_MATRIX)

        auxiliary_vars = model.addVars(
            len(factual_impacts), obj=factual_impacts, vtype=gb.GRB.CONTINUOUS
        )
        model.update()

        for j, intervention in enumerate(POSSIBLE_INTERVENTIONS_MATRIX):
            for k, neighbor in enumerate(neighbor_index_matrix[index]):
                if intervention[k] == 1:
                    model.addConstr(auxiliary_vars[j] <= interventions[neighbor])
                else:
                    model.addConstr(auxiliary_vars[j] <= 1 - interventions[neighbor])
        model.addConstr(sum(auxiliary_vars.values()) == 1)

        if tau_value is not None:
            for group_idx in range(A_frac.shape[1]):
                group_impact_diff = calculate_all_possible_impacts(index, np.eye(A_frac.shape[1])[group_idx], POSSIBLE_INTERVENTIONS_MATRIX) - factual_impacts
                model.addConstr(
                    sum(auxiliary_vars[j] * group_impact_diff[j] for j in range(len(factual_impacts))) <= tau_value
                )

    for index in range(n):
        add_auxiliary_constraints(index)

    model.setObjective(model.getObjective(), gb.GRB.MAXIMIZE)
    model.optimize()

    if model.status == gb.GRB.OPTIMAL:
        return np.array([interventions[i].X for i in range(n)]).astype(bool)
    else:
        raise RuntimeError("Optimization failed.")

# Run optimization for each tau value
for tau_value in TAU_VALUES:
    try:
        optimal_interventions = optimize_interventions(tau_value, A_frac, POSSIBLE_INTERVENTIONS_MATRIX)
        print(f"Optimal interventions: {np.where(optimal_interventions)}")
    except RuntimeError as e:
        print(f"Optimization failed for tau={tau_value}: {e}")


Running optimization for tau=0.566
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7662 64-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 128 physical cores, 256 logical processors, using up to 32 threads

Academic license 2586688 - for non-commercial use only - registered to ru___@ucsd.edu
Optimize a model with 192081 rows, 31850 columns and 627626 nonzeros
Model fingerprint: 0xa5000302
Variable types: 31360 continuous, 490 integer (490 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [6e-02, 6e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-01, 1e+02]
Presolve removed 1051 rows and 0 columns
Presolve time: 0.24s

Explored 0 nodes (0 simplex iterations) in 0.34 seconds (0.37 work units)
Thread count was 1 (of 256 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -
Optimization failed for tau=0.566: Optimizat